# Wrangling University of British Columbia Survey Data
## A Particularly Spooky Halloween Dataset
### James Davis
Table of contents:
* [Getting an overview of the data.](#overview)
* [Checking the Internal ID column for duplicates.](#id_check)
* [Cleaning age values.](#age_clean)
* [Dropping unnecessary columns.](#col_drop)
* [Combining the four media columns.](#media)
* [Deleting abandoned responses.](#del_aban)
* [Filling invalid categorical fields.](#inv_fields)
* [Converting each of the three categories of opinion on each lolly to an integer.](#lolly_int)
* [Standardising countries](#countries)
* [Standardising states](#states)
* ['Unpivoting' the responses.](#unpivot)
* [Final checks and cleanup before saving output.](#output)


In [75]:
import pandas as pd
import numpy as np
import os

In [76]:
pwd = os.getcwd() # Helps with file management.
survey_df = pd.read_excel(pwd + '\data1_raw_survey_input.xlsx')

D:\Personal\Education\Programming\Anaconda1\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


### Data Overview <a class="anchor" id="overview"></a>

In [148]:
pd.options.display.max_rows = 30
survey_df.head()

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q8: DESPAIR OTHER,Q9: OTHER COMMENTS,Q10: DRESS,Unnamed: 113,Q11: DAY,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo],"Click Coordinates (x, y)"
0,90258773,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,NaN,Bottom line is Twix is really the only candy w...,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(84, 25)"
2,90272829,NaN,Male,49,USA,Virginia,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,NaN,Raisins can go to hell,White and gold,NaN,Sunday,NaN,1.0,NaN,NaN,"(75, 23)"
4,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,NaN,NaN,White and gold,NaN,Friday,NaN,1.0,NaN,NaN,"(70, 10)"


In [78]:
survey_df.dtypes

Internal ID                   int64
Q1: GOING OUT?               object
Q2: GENDER                   object
Q3: AGE                      object
Q4: COUNTRY                  object
                             ...   
Q12: MEDIA [Daily Dish]     float64
Q12: MEDIA [Science]        float64
Q12: MEDIA [ESPN]           float64
Q12: MEDIA [Yahoo]          float64
Click Coordinates (x, y)     object
Length: 120, dtype: object

It'll be worth converting all these once we've finished surgically alterting this dataset.

### Checking for Duplicate Respondents <a class="anchor" id="id_check"></a>
Never trust an assumption about your data until you've verified it, especially when it's this zany!

In [79]:
print(f'Total IDs: {len(survey_df["Internal ID"])}')
print(f'Unique IDs: {survey_df["Internal ID"].nunique()}')

Total IDs: 2460
Unique IDs: 2460


### Clean Age Values <a class="anchor" id="age_clean"></a>
The data contain a few... aberrant responses.

Note: I'll be making a few copies throughout so it's easier to track my progress at different stages and roll back mistakes. 
The tradeoff is storing a bunch of variables, but for a project of this size it's not a real concern.

In [80]:
survey_df_agedrop = survey_df.copy()

In [81]:
print(f'Null values: {survey_df_agedrop["Q3: AGE"].isna().sum()}')
print(f'Non-numeric values: {len([age for age in survey_df_agedrop["Q3: AGE"] if isinstance(age, str) == True])}')
print(f'Total values: {len(survey_df_agedrop["Q3: AGE"])}')

Null values: 84
Non-numeric values: 24
Total values: 2460


We therefore need 108 cells to be 0 whilst preserving the int64 dtype.

In [82]:
survey_df_agedrop['Q3: AGE'] = pd.to_numeric(survey_df_agedrop['Q3: AGE'], errors='coerce')
survey_df_agedrop.replace(np.nan, 0, inplace=True)
survey_df_agedrop.astype({'Q3: AGE': 'int64'}, errors='ignore', copy=False)
survey_df_agedrop['Q3: AGE'].value_counts()

0.0       108
40.0       92
34.0       90
37.0       89
43.0       86
         ... 
88.0        1
312.0       1
70.5        1
99.0        1
1000.0      1
Name: Q3: AGE, Length: 84, dtype: int64

In [83]:
print(f'Null values: {survey_df_agedrop["Q3: AGE"].isna().sum()}')
print(f'Non-numeric values: {len([age for age in survey_df_agedrop["Q3: AGE"] if isinstance(age, str) == True])}')
print(len(survey_df_agedrop['Q3: AGE']))

Null values: 0
Non-numeric values: 0
2460


Now all the rubbish has been swept out, time for a sense-check. It's very unlikely that a 300-year-old is taking this survey. I'm going to assume anyone aged 90 or over is just trying to be funny.

In [84]:
print(f'Number of dubious ages: {len([age for age in survey_df_agedrop["Q3: AGE"] if age >= 90])}')

Number of dubious ages: 8


In [85]:
survey_df_sensecheck = survey_df_agedrop.copy()

In [86]:
survey_df_sensecheck["Q3: AGE"] = [int(age) if age < 90 else 0 for age in survey_df_sensecheck["Q3: AGE"]]
print(f'Number of dubious ages: {len([age for age in survey_df_sensecheck["Q3: AGE"] if age >= 90])}')

Number of dubious ages: 0


I'd also like to exclude a few exceptionally young ages. Halloween lollies are the domain of kids, but I'd say it's highly unlikely any kid under three is capable of coherent opinions. If you were an opinionated two-year-old, don't hesitate to refrain from letting me know.

In [87]:
print(f'Number of far-too-young ages: {len([age for age in survey_df_sensecheck["Q3: AGE"] if age <= 2 and age != 0])}')

Number of far-too-young ages: 1


In [88]:
survey_df_sensecheck["Q3: AGE"] = [int(age) if age > 2 else 0 for age in survey_df_sensecheck["Q3: AGE"]]
print(f'Number of far-too-young ages: {len([age for age in survey_df_sensecheck["Q3: AGE"] if age <= 2 and age != 0])}')

Number of far-too-young ages: 0


To finish up, I'm going to replace all 0s with the median age.

In [89]:
survey_df_sensecheck["Q3: AGE"] = [age if age > 0 else survey_df_sensecheck["Q3: AGE"].median() for age in survey_df_sensecheck["Q3: AGE"]]
survey_df_sensecheck["Q3: AGE"] = survey_df_sensecheck["Q3: AGE"].astype('int64')
survey_df_sensecheck["Q3: AGE"].value_counts()

41    191
40     92
34     90
37     89
43     86
     ... 
8       2
4       1
88      1
74      1
77      1
Name: Q3: AGE, Length: 74, dtype: int64

### Dropping Unneeded Columns <a class="anchor" id="col_drop"></a>
There are several suspect columns in this dataset that are unlikely to be valuable without the aid of quantum computing.

In [90]:
survey_df_sensecheck.columns

Index(['Internal ID', 'Q1: GOING OUT?', 'Q2: GENDER', 'Q3: AGE', 'Q4: COUNTRY',
       'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       ...
       'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS', 'Q10: DRESS', 'Unnamed: 113',
       'Q11: DAY', 'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]',
       'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]', 'Click Coordinates (x, y)'],
      dtype='object', length=120)

In [91]:
survey_df_sensecheck['Unnamed: 113'].value_counts()

0                                                                          2451
dress (https://survey.ubc.ca/media/assets/user/14372/storage/dress.png)       9
Name: Unnamed: 113, dtype: int64

While some of these columns are just strictly useless, others could conceivably be useful. For instance, knowing the day someone completed the survey could be part of a meta study on when people are most likely to do surveys. But that's not very useful here.

In [92]:
survey_df_coldrops = survey_df_sensecheck.copy()
survey_df_coldrops.drop(columns=['Q7: JOY OTHER', 'Q8: DESPAIR OTHER', 'Q9: OTHER COMMENTS', 'Unnamed: 113', 'Q10: DRESS', 'Click Coordinates (x, y)', 'Q11: DAY'], inplace=True)
survey_df_coldrops.columns

Index(['Internal ID', 'Q1: GOING OUT?', 'Q2: GENDER', 'Q3: AGE', 'Q4: COUNTRY',
       'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q6 | 100 Grand Bar',
       'Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes)',
       'Q6 | Any full-sized candy bar', 'Q6 | Black Jacks',
       ...
       'Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein',
       'Q6 | Vicodin', 'Q6 | Whatchamacallit Bars', 'Q6 | White Bread',
       'Q6 | Whole Wheat anything', 'Q6 | York Peppermint Patties',
       'Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]',
       'Q12: MEDIA [Yahoo]'],
      dtype='object', length=113)

### Combine Media Columns <a class="anchor" id="media"></a>
The media columns are a highly-unrelated portion of the survey where respondents are presented with four mobile homepages and asked to honestly select which one they'd click on. 

The reason we're keeping this and not the dress, click coordinates or day columns is because it looks like a neat little challenge. Sue me.

I want a single media column containing a categorical variable for each of the media organisations.

In [93]:
survey_df_media = survey_df_coldrops.copy()

I'm thinking I handle this via the following process:
1. Change media column values to 1 - 4, where 2 would be 'Science', 3 would be 'ESPN' etc.
2. Collapse all values into new MEDIA column.
3. Convert values 1 - 4 into 'Daily Dish' - 'Yahoo' respectively.
4. Drop all four original media columns.

If any of these clowns have selected multiple answers, we'll know soon enough!

In [94]:
def num_changer(data, numadd):
    result = [i + numadd if i > 0 else i for i in data]
    return result

In [95]:
media_data = ['Q12: MEDIA [Daily Dish]', 'Q12: MEDIA [Science]', 'Q12: MEDIA [ESPN]', 'Q12: MEDIA [Yahoo]']

for ind, media_col in enumerate(media_data):
    survey_df_media[media_col] = num_changer(survey_df_media[media_col], ind)

In [96]:
survey_df_media[media_data].loc[1770:1790] # I've taken this oddball slice because it contains responses in each column.

,Q12: MEDIA [Daily Dish],Q12: MEDIA [Science],Q12: MEDIA [ESPN],Q12: MEDIA [Yahoo]
1770,0.0,0.0,0.0,0.0
1771,0.0,0.0,0.0,0.0
1772,0.0,0.0,0.0,4.0
1773,0.0,2.0,0.0,0.0
1774,0.0,2.0,0.0,0.0
1775,0.0,0.0,0.0,0.0
1776,0.0,2.0,0.0,0.0
1777,0.0,0.0,0.0,0.0
1778,0.0,2.0,0.0,0.0
1779,0.0,2.0,0.0,0.0


In [97]:
survey_df_media['Q12: MEDIA'] = survey_df_media[media_data].sum(axis=1)
survey_df_media['Q12: MEDIA'].loc[1770:1790]

1770    0.0
1771    0.0
1772    4.0
1773    2.0
1774    2.0
1775    0.0
1776    2.0
1777    0.0
1778    2.0
1779    2.0
1780    0.0
1781    0.0
1782    0.0
1783    2.0
1784    1.0
1785    0.0
1786    2.0
1787    3.0
1788    0.0
1789    0.0
1790    2.0
Name: Q12: MEDIA, dtype: float64

In [98]:
valuedict = {0: 'None',
             1: 'Daily Dish',
             2: 'Science',
             3: 'ESPN',
             4: 'Yahoo'}

for key, value in valuedict.items():
    for ind, media in enumerate(survey_df_media['Q12: MEDIA']):
        if media == key:
            survey_df_media.loc[ind, 'Q12: MEDIA'] = value

survey_df_media.loc[1770:1790, 'Q12: MEDIA']

1770          None
1771          None
1772         Yahoo
1773       Science
1774       Science
1775          None
1776       Science
1777          None
1778       Science
1779       Science
1780          None
1781          None
1782          None
1783       Science
1784    Daily Dish
1785          None
1786       Science
1787          ESPN
1788          None
1789          None
1790       Science
Name: Q12: MEDIA, dtype: object

In [99]:
survey_df_media.drop(columns=media_data, inplace=True)

In [100]:
survey_df_media['Q12: MEDIA'].value_counts()

Science       1362
None           847
ESPN            99
Daily Dish      85
Yahoo           67
Name: Q12: MEDIA, dtype: int64

### Delete Abandoned Responses <a class="anchor" id="del_aban"></a>
Some users opened the survey, but immediately closed the tab. We'll delete these responses by scanning each row for responses of '0' in all main question fields, getting the index of each, then dropping them.

In [101]:
rowdrops = (survey_df_media.iloc[:, 6:111:1] == 0) # Returns a dataframe of True/ False values, where True = 0.
dropindex = rowdrops.index[rowdrops['Q6 | 100 Grand Bar']].tolist()
dropindex[0:5]

[0, 2, 6, 10, 18]

In [102]:
survey_df_abandrops = survey_df_media.copy()
survey_df_abandrops.drop(dropindex, inplace=True)
survey_df_abandrops.reset_index(inplace=True)
survey_df_abandrops.drop('index', axis=1, inplace=True)

In [103]:
survey_df_abandrops.head(5)

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q12: MEDIA
0,90272821,No,Male,44,USA,NM,MEH,DESPAIR,JOY,MEH,...,JOY,DESPAIR,JOY,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Science
1,90272840,No,Male,40,us,or,MEH,DESPAIR,JOY,MEH,...,JOY,MEH,JOY,DESPAIR,JOY,JOY,DESPAIR,DESPAIR,DESPAIR,Science
2,90272841,No,Male,23,usa,exton pa,JOY,DESPAIR,JOY,DESPAIR,...,JOY,DESPAIR,JOY,MEH,JOY,JOY,DESPAIR,DESPAIR,JOY,Science
3,90272852,No,Male,41,0,0,JOY,DESPAIR,JOY,0,...,JOY,MEH,JOY,DESPAIR,DESPAIR,JOY,DESPAIR,DESPAIR,JOY,Science
4,90272854,No,Male,33,canada,ontario,JOY,DESPAIR,JOY,DESPAIR,...,MEH,DESPAIR,JOY,JOY,MEH,DESPAIR,DESPAIR,DESPAIR,DESPAIR,Science


### Filling Invalid Categorical Fields <a class="anchor" id="inv_fields"></a>
There are only two valid choices ('Yes' and 'No'). I'm going to assume any response of 0 means they'd rather not say. This makes it consistent with the gender field, but even that has a similar issue.

In [104]:
survey_df_abandrops[['Q1: GOING OUT?', 'Q2: GENDER']].value_counts()

Q1: GOING OUT?  Q2: GENDER        
No              Male                  860
                Female                493
Yes             Male                  137
                Female                 82
No              I'd rather not say     48
0               Male                   44
No              Other                  18
0               Female                  8
Yes             I'd rather not say      8
0               0                       4
                I'd rather not say      4
No              0                       3
Yes             Other                   3
0               Other                   1
dtype: int64

In [105]:
problemcols = ['Q1: GOING OUT?', 'Q2: GENDER']

for column in problemcols:
    survey_df_abandrops[column] = [answer if answer != 0 else "I'd rather not say" for answer in survey_df_abandrops[column]]

In [106]:
survey_df_abandrops[['Q1: GOING OUT?', 'Q2: GENDER']].value_counts()

Q1: GOING OUT?      Q2: GENDER        
No                  Male                  860
                    Female                493
Yes                 Male                  137
                    Female                 82
No                  I'd rather not say     51
I'd rather not say  Male                   44
No                  Other                  18
I'd rather not say  Female                  8
                    I'd rather not say      8
Yes                 I'd rather not say      8
                    Other                   3
I'd rather not say  Other                   1
dtype: int64

### Converting Lolly Categories <a class="anchor" id="lolly_int"></a>
'Despair', 'Meh' and 'Joy' are reasonably functional, but if I were hired to clean this data set knowing there'd be further exploration, I'd want a slightly better way of scoring and aggregating this information. Just having them as integers seems like a solid idea. We can use some of the tricks employed earlier to convert these values to integers.

In [107]:
survey_df_cats = survey_df_abandrops.copy()

In [108]:
scoredict = {'DESPAIR': 1,
             'MEH': 2,
             'JOY': 3}

for column in survey_df_cats:
    for key, value in scoredict.items():
        survey_df_cats[column] = [answer if key != answer else value for answer in survey_df_cats[column]]

In [109]:
survey_df_cats.head(10)

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q12: MEDIA
0,90272821,No,Male,44,USA,NM,2,1,3,2,...,3,1,3,1,1,1,1,1,1,Science
1,90272840,No,Male,40,us,or,2,1,3,2,...,3,2,3,1,3,3,1,1,1,Science
2,90272841,No,Male,23,usa,exton pa,3,1,3,1,...,3,1,3,2,3,3,1,1,3,Science
3,90272852,No,Male,41,0,0,3,1,3,0,...,3,2,3,1,1,3,1,1,3,Science
4,90272854,No,Male,33,canada,ontario,3,1,3,1,...,2,1,3,3,2,1,1,1,1,Science
5,90272858,No,Male,40,Canada,Ontario,3,1,3,2,...,3,1,3,2,1,2,1,1,1,Science
6,90272859,No,Female,53,Us,Wa,2,1,3,2,...,3,2,3,1,1,2,1,1,2,Science
7,90272865,No,Male,56,Canada,Quebec,3,2,3,2,...,3,2,2,1,2,2,3,1,2,Science
8,90272866,No,Male,64,US,NY,2,2,3,2,...,3,1,2,3,3,1,1,1,2,Science
9,90272867,Yes,Male,43,Murica,California,3,1,3,2,...,3,1,3,1,1,3,1,1,3,Science


### Cleaning Country Fields <a class="anchor" id="countries"></a>
Now for a real challenge! These fields are absolutely abyssmal, filled with inconsistent capitalisation, abbreviations and colloquialisms (such as 'Murica' or 'Merica'). This means we can't just throw a dictionary at it. Let's start with the country field.

In [110]:
survey_df_cats['Q4: COUNTRY'].value_counts()

USA              515
United States    379
usa              158
Canada           104
US                98
                ... 
Canae              1
USSA               1
Narnia             1
United Statss      1
'merica            1
Name: Q4: COUNTRY, Length: 95, dtype: int64

Yikes! OK, Here's the current plan:
1. Make everything upper case to cut a few inconsistencies in a broad sweep.
2. Cut all punctuation and spaces from each country.
3. Paste in a dictionary of ISO3166 country codes. Source: https://gist.github.com/carlopires/1261951/d13ca7320a6abcd4b0aa800d351a31b54cefdff4
4. Make the dictionary all uppercase.
5. Remove all spaces from existing dictionary values.
6. Append other common terms to dictionary values (eg 'US').
7. Loop through the dictionary and replace each value with ISO country codes, taking care to use '==' instead of 'in' to account for cases where new values exist in other countries (eg 'US' can be found in the word 'AUSTRALIA', so we need an exact match).
8. Celebrate.

In [111]:
survey_df_countryfix = survey_df_cats.copy()

In [112]:
import string

survey_df_countryfix['Q4: COUNTRY'] = [str(country).upper() for country in survey_df_countryfix['Q4: COUNTRY']]

for ind, country in enumerate(survey_df_countryfix['Q4: COUNTRY']):
    survey_df_countryfix.loc[ind, 'Q4: COUNTRY'] = ''.join([char.replace(' ', '') for char in survey_df_countryfix['Q4: COUNTRY'].iloc[ind] if char not in string.punctuation])

In [113]:
from data2_isodict import ISO3166

countrydict = ISO3166.copy()
sanitised_countries = [str(country).upper().replace(' ', '') for country in countrydict.values()]

In [114]:
for ind, key in enumerate(countrydict.keys()):
    countrydict[key] = sanitised_countries[ind]

# Given the data is static, I don't mind manually updating the dictionary with a few synonyms it missed.
countrydict['US'] = ['UNITEDSTATESOFAMERICA', 'UNITEDSTATES', 'USA', 'US', 'THEUNITEDSTATES', 'AMERICA']
countrydict['GB'] = ['SCOTLAND', 'UK', 'UNITEDKINGDOM']
countrydict['KR'] = ['SOUTHKOREA', 'REPUBLICOFSOUTHKOREA', 'KOREA']
countrydict['AE'] = 'UAE'
countrydict['TW'] = ['TAIWANPROVINCEOFCHINA', 'TAIWAN']

In [115]:
countrydict['US']

['UNITEDSTATESOFAMERICA',
 'UNITEDSTATES',
 'USA',
 'US',
 'THEUNITEDSTATES',
 'AMERICA']

In [116]:
for ind, country in enumerate(survey_df_countryfix['Q4: COUNTRY']):
    for key, values in countrydict.items():
        if isinstance(values, list) == True:
            for value in values:
                if value == country:
                    survey_df_countryfix.loc[ind, 'Q4: COUNTRY'] = key
        else:
            if values == country:
                survey_df_countryfix.loc[ind, 'Q4: COUNTRY'] = key

In [117]:
survey_df_countryfix['Q4: COUNTRY'].value_counts()

US               1475
CA                129
0                  20
GB                 15
DE                  5
                 ... 
ENDLAND             1
NORTHCAROLINA       1
CR                  1
UNITEDSTAES         1
GR                  1
Name: Q4: COUNTRY, Length: 59, dtype: int64

In [118]:
failures = len([country for country in survey_df_countryfix['Q4: COUNTRY'] if len(country) > 2])
successes = len([country for country in survey_df_countryfix['Q4: COUNTRY'] if len(country) == 2])

print(f"Failed ISO conversions: {failures}")
print(f"Successful ISO conversions: {successes}")
print(f"{round((failures/ successes) * 100, 2)}% fail rate.")

Failed ISO conversions: 41
Successful ISO conversions: 1651
2.48% fail rate.


Decent! Given this is a static data set that won't be run again, I'm OK with having slapped a variety of synonyms into a dictionary.

Let's now get the index of each trouble row and replace each one with 'US', because that has the highest likelihood of being correct given Americans are vastly overrepresented in the results already. 

In [119]:
countrydrop = [ind for ind, country in enumerate(survey_df_countryfix['Q4: COUNTRY']) if len(country) > 2 or len(country) < 2]
survey_df_countryfix['Q4: COUNTRY'] = survey_df_countryfix['Q4: COUNTRY'].drop(countrydrop)
survey_df_countryfix['Q4: COUNTRY'].replace({np.nan: 'US', '0': 'US'}, inplace=True)

In [120]:
survey_df_countryfix['Q4: COUNTRY'].value_counts()

US    1537
CA     129
GB      15
DE       5
IE       3
JP       3
NL       3
DK       2
KR       2
FR       2
CH       2
MX       2
AU       1
CN       1
CR       1
IS       1
TW       1
AE       1
ES       1
GR       1
Name: Q4: COUNTRY, dtype: int64

### Cleaning State Fields <a class="anchor" id="states"></a>
Our ol' faithful dictionary-based solution gets markedly less efficient here. Every country has numerous states, all with their own abbreviations, slang and ways for a wayward survey-doer to go awry. 

In [121]:
survey_df_countryfix['Q5: STATE, PROVINCE, COUNTY, ETC'].value_counts()

California       104
CA                62
Texas             43
Illinois          42
Oregon            38
                ... 
Ri                 1
indiana            1
New Hampshire      1
calif              1
in                 1
Name: Q5: STATE, PROVINCE, COUNTY, ETC, Length: 412, dtype: int64

In [122]:
survey_df_statefix = survey_df_countryfix.copy()

This makes each state upper case, strips them of punctuation and removes spaces.

In [123]:
survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'] = [str(country).upper() for country in survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC']]

for ind, country in enumerate(survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC']):
    survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC'] = ''.join([char.replace(' ', '') for char in survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC'] if char not in string.punctuation])

In [124]:
survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'].value_counts()

CALIFORNIA                          134
CA                                   83
ILLINOIS                             55
TEXAS                                50
ONTARIO                              47
                                   ... 
OAKLANDCOUNTYMICHIGAN                 1
OBLIVIONIFTHINGSKEEPGOINGTHISWAY      1
TEHAMA                                1
NEWFOUNDLANDANDLABRADOR               1
RAPPAHANNOCKCOUNTYVA                  1
Name: Q5: STATE, PROVINCE, COUNTY, ETC, Length: 258, dtype: int64

We'll use pycountry's subdivision object to iterate through our states and make them consistent. The final output will be state names. Process:
1. Create list of subdivision objects containing state codes and names.
2. Extract state code, country code and state name, then put them into a dictionary. Altering subdivision list properties can have unintended consequences on all instances, even if you deep copy the list, which is why I'm going to the trouble of housing them in a different object.
2. Create separate list of sanitised state names (i.e. all caps, no punctuation or spaces), then replace the ones in the dictionary with those.
3. Loop through each state in the dataframe, replacing them with codes from the dictionary.
4. Use codes to insert original subdivision state names.

In [125]:
import pycountry as pc

original_subdivs = list(pc.subdivisions)
original_subdivs[0] # What a subdivision looks like. Various methods access each component.

Subdivision(code='AD-02', country_code='AD', name='Canillo', parent_code=None, type='Parish')

In [126]:
subdiv_dict = dict()
for ind, state in enumerate(original_subdivs):
    subdiv_dict.update({original_subdivs[ind].code: [original_subdivs[ind].country.alpha_2, original_subdivs[ind].name]})

In [127]:
subdiv_dict

{'AD-02': ['AD', 'Canillo'],
 'AD-03': ['AD', 'Encamp'],
 'AD-04': ['AD', 'La Massana'],
 'AD-05': ['AD', 'Ordino'],
 'AD-06': ['AD', 'Sant Julià de Lòria'],
 'AD-07': ['AD', 'Andorra la Vella'],
 'AD-08': ['AD', 'Escaldes-Engordany'],
 'AE-AJ': ['AE', "'Ajmān"],
 'AE-AZ': ['AE', 'Abū Ȥaby [Abu Dhabi]'],
 'AE-DU': ['AE', 'Dubayy'],
 'AE-FU': ['AE', 'Al Fujayrah'],
 'AE-RK': ['AE', 'Ra’s al Khaymah'],
 'AE-SH': ['AE', 'Ash Shāriqah'],
 'AE-UQ': ['AE', 'Umm al Qaywayn'],
 'AF-BAL': ['AF', 'Balkh'],
 'AF-BAM': ['AF', 'Bāmyān'],
 'AF-BDG': ['AF', 'Bādghīs'],
 'AF-BDS': ['AF', 'Badakhshān'],
 'AF-BGL': ['AF', 'Baghlān'],
 'AF-DAY': ['AF', 'Dāykundī'],
 'AF-FRA': ['AF', 'Farāh'],
 'AF-FYB': ['AF', 'Fāryāb'],
 'AF-GHA': ['AF', 'Ghaznī'],
 'AF-GHO': ['AF', 'Ghōr'],
 'AF-HEL': ['AF', 'Helmand'],
 'AF-HER': ['AF', 'Herāt'],
 'AF-JOW': ['AF', 'Jowzjān'],
 'AF-KAB': ['AF', 'Kābul'],
 'AF-KAN': ['AF', 'Kandahār'],
 'AF-KAP': ['AF', 'Kāpīsā'],
 'AF-KDZ': ['AF', 'Kunduz'],
 'AF-KHO': ['AF', 'Khōst'],

In [128]:
subdiv_dict_caps = subdiv_dict.copy()

In [129]:
for key, values in subdiv_dict_caps.items():
    subdiv_dict_caps.update({key: [values[0], ''.join([char.replace(' ', '').upper() for char in values[1] if char not in string.punctuation])]})

In [130]:
pd.options.display.max_rows = 30 # Adjust at your leisure.
subdiv_dict_caps

{'AD-02': ['AD', 'CANILLO'],
 'AD-03': ['AD', 'ENCAMP'],
 'AD-04': ['AD', 'LAMASSANA'],
 'AD-05': ['AD', 'ORDINO'],
 'AD-06': ['AD', 'SANTJULIÀDELÒRIA'],
 'AD-07': ['AD', 'ANDORRALAVELLA'],
 'AD-08': ['AD', 'ESCALDESENGORDANY'],
 'AE-AJ': ['AE', 'AJMĀN'],
 'AE-AZ': ['AE', 'ABŪȤABYABUDHABI'],
 'AE-DU': ['AE', 'DUBAYY'],
 'AE-FU': ['AE', 'ALFUJAYRAH'],
 'AE-RK': ['AE', 'RA’SALKHAYMAH'],
 'AE-SH': ['AE', 'ASHSHĀRIQAH'],
 'AE-UQ': ['AE', 'UMMALQAYWAYN'],
 'AF-BAL': ['AF', 'BALKH'],
 'AF-BAM': ['AF', 'BĀMYĀN'],
 'AF-BDG': ['AF', 'BĀDGHĪS'],
 'AF-BDS': ['AF', 'BADAKHSHĀN'],
 'AF-BGL': ['AF', 'BAGHLĀN'],
 'AF-DAY': ['AF', 'DĀYKUNDĪ'],
 'AF-FRA': ['AF', 'FARĀH'],
 'AF-FYB': ['AF', 'FĀRYĀB'],
 'AF-GHA': ['AF', 'GHAZNĪ'],
 'AF-GHO': ['AF', 'GHŌR'],
 'AF-HEL': ['AF', 'HELMAND'],
 'AF-HER': ['AF', 'HERĀT'],
 'AF-JOW': ['AF', 'JOWZJĀN'],
 'AF-KAB': ['AF', 'KĀBUL'],
 'AF-KAN': ['AF', 'KANDAHĀR'],
 'AF-KAP': ['AF', 'KĀPĪSĀ'],
 'AF-KDZ': ['AF', 'KUNDUZ'],
 'AF-KHO': ['AF', 'KHŌST'],
 'AF-KNR': ['AF', 

In [131]:
# This takes 2 - 3 minutes to run.
for ind, state in enumerate(survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC']):
    for key, values in subdiv_dict_caps.items():
        if values[0] in survey_df_statefix.loc[ind, 'Q4: COUNTRY']:
            if values[1] in survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC']:
                survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC'] = key

In [132]:
failrate = round((len([dudstate for dudstate in survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'] if len(dudstate) > 6])/len(survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'])) * 100, 2)

print(f'{failrate}% of states were not captured.')

3.74% of states were not captured.


In [133]:
survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'].value_counts(dropna=False)

US-CA                   148
CA                       83
US-IL                    56
US-WA                    52
US-TX                    51
                       ... 
SANJOSE                   1
MT                        1
WASPOKANE                 1
SOUTHHOLLAND              1
RAPPAHANNOCKCOUNTYVA      1
Name: Q5: STATE, PROVINCE, COUNTY, ETC, Length: 209, dtype: int64

It looks like a few states didn't quite convert to the country-state code pairings we need, CA being a big one. We can search for these rogue codes within our dictionary keys (country-state pairings) to capture the rest.

In [134]:
# This takes 2 - 3 minutes to run.
for ind, state in enumerate(survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC']):
    for key, values in subdiv_dict.items():
        if values[0] in survey_df_statefix.loc[ind, 'Q4: COUNTRY']:
            if survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC'] in key:
                survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC'] = values[1]

In [135]:
pd.options.display.max_rows = 30
survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'].value_counts()

California              231
Washington               95
New York                 81
Illinois                 76
Texas                    69
                       ... 
Midlothian                1
KENT                      1
ORANGECOUNTYCA            1
Devon                     1
RAPPAHANNOCKCOUNTYVA      1
Name: Q5: STATE, PROVINCE, COUNTY, ETC, Length: 166, dtype: int64

We will now impute all unrecognised values as 'Unknown', then promptly slap whoever set up the survey for making all this necessary.

Note: While these unrecognised values are a combination of nulls and joke answers, we are losing a portion of legitimate responses. Capturing the few loose strands of legitimacy would unfortunately be a huge timesink, and there aren't enough to warrant a complex separate solution.

In [136]:
null_values = len([state for state in survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'] if state == '0'])
uncaptured_values = len([state for state in survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'] if state.isupper() == True])

print(f'{null_values} nulls\n{uncaptured_values} uncaptured values')

33 nulls
86 uncaptured values


In [137]:
for ind, state in enumerate(survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC']):
    if state.isupper() == True or state == '0':
        survey_df_statefix.loc[ind, 'Q5: STATE, PROVINCE, COUNTY, ETC'] = 'Unknown'

In [138]:
null_values = len([state for state in survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'] if state == '0'])
uncaptured_values = len([state for state in survey_df_statefix['Q5: STATE, PROVINCE, COUNTY, ETC'] if state.isupper() == True])

print(f'{null_values} nulls\n{uncaptured_values} uncaptured values')

0 nulls
0 uncaptured values


### Unpivoting Q6 <a class="anchor" id="unpivot"></a>
Perhaps the worst eyesore in the whole dataset is how question six gingerly manspreads across 100+ columns.

Our final major task will be to unpivot, or 'melt', these columns so they run vertically rather than horziontally. This will make the data easier to query later.

In [139]:
survey_df_statefix.head(3)

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q6 | 100 Grand Bar,Q6 | Anonymous brown globs that come in black and orange wrappers\t(a.k.a. Mary Janes),Q6 | Any full-sized candy bar,Q6 | Black Jacks,...,Q6 | Tolberone something or other,Q6 | Trail Mix,Q6 | Twix,"Q6 | Vials of pure high fructose corn syrup, for main-lining into your vein",Q6 | Vicodin,Q6 | Whatchamacallit Bars,Q6 | White Bread,Q6 | Whole Wheat anything,Q6 | York Peppermint Patties,Q12: MEDIA
0,90272821,No,Male,44,US,New Mexico,2,1,3,2,...,3,1,3,1,1,1,1,1,1,Science
1,90272840,No,Male,40,US,Oregon,2,1,3,2,...,3,2,3,1,3,3,1,1,1,Science
2,90272841,No,Male,23,US,Unknown,3,1,3,1,...,3,1,3,2,3,3,1,1,3,Science


But we first need to bring the media column up...

In [140]:
media_col = survey_df_statefix.pop('Q12: MEDIA')
survey_df_statefix.insert(6, 'Q12: MEDIA', media_col)


...then remove the 'Q6' part of each lolly preference.

In [141]:
for ind, row in enumerate(survey_df_statefix.loc[:, 'Q6 | 100 Grand Bar':]):
    survey_df_statefix.columns = [row.replace('Q6 | ', '') for row in survey_df_statefix.columns]

Now it's time for the magic.

In [142]:
survey_df_unpivoted = survey_df_statefix.melt(id_vars=['Internal ID', 'Q1: GOING OUT?',	'Q2: GENDER', 'Q3: AGE', 'Q4: COUNTRY',	'Q5: STATE, PROVINCE, COUNTY, ETC', 'Q12: MEDIA'], var_name='Topic', value_name='Rating')

In [143]:
survey_df_unpivoted

,Internal ID,Q1: GOING OUT?,Q2: GENDER,Q3: AGE,Q4: COUNTRY,"Q5: STATE, PROVINCE, COUNTY, ETC",Q12: MEDIA,Topic,Rating
0,90272821,No,Male,44,US,New Mexico,Science,100 Grand Bar,2
1,90272840,No,Male,40,US,Oregon,Science,100 Grand Bar,2
2,90272841,No,Male,23,US,Unknown,Science,100 Grand Bar,3
3,90272852,No,Male,41,US,Unknown,Science,100 Grand Bar,3
4,90272854,No,Male,33,CA,Ontario,Science,100 Grand Bar,3
...,...,...,...,...,...,...,...,...,...
176434,90314022,No,Female,26,US,Michigan,Science,York Peppermint Patties,3
176435,90314359,No,Male,24,US,Maryland,None,York Peppermint Patties,2
176436,90314580,No,Female,33,US,New York,Science,York Peppermint Patties,3
176437,90314634,No,Female,26,US,Tennessee,Science,York Peppermint Patties,2


### Cleaning Up & Saving the Data <a class="anchor" id="cleanup"></a>

Now it's time to correct any errant datatypes, rename the columns and save the output.

In [144]:
survey_df_unpivoted.dtypes

Internal ID                          int64
Q1: GOING OUT?                      object
Q2: GENDER                          object
Q3: AGE                              int64
Q4: COUNTRY                         object
Q5: STATE, PROVINCE, COUNTY, ETC    object
Q12: MEDIA                          object
Topic                               object
Rating                               int64
dtype: object

In [145]:
survey_df_unpivoted.rename(mapper={'Q1: GOING OUT?': 'Halloween Plans?', 'Q2: GENDER': 'Gender', 'Q3: AGE': 'Age', 'Q4: COUNTRY': 'Country',
       'Q5: STATE, PROVINCE, COUNTY, ETC':'State', 'Q12: MEDIA':'Media Preference'}, axis=1, inplace=True)

In [146]:
survey_df_unpivoted

,Internal ID,Halloween Plans?,Gender,Age,Country,State,Media Preference,Topic,Rating
0,90272821,No,Male,44,US,New Mexico,Science,100 Grand Bar,2
1,90272840,No,Male,40,US,Oregon,Science,100 Grand Bar,2
2,90272841,No,Male,23,US,Unknown,Science,100 Grand Bar,3
3,90272852,No,Male,41,US,Unknown,Science,100 Grand Bar,3
4,90272854,No,Male,33,CA,Ontario,Science,100 Grand Bar,3
...,...,...,...,...,...,...,...,...,...
176434,90314022,No,Female,26,US,Michigan,Science,York Peppermint Patties,3
176435,90314359,No,Male,24,US,Maryland,None,York Peppermint Patties,2
176436,90314580,No,Female,33,US,New York,Science,York Peppermint Patties,3
176437,90314634,No,Female,26,US,Tennessee,Science,York Peppermint Patties,2


In [147]:
survey_df_unpivoted.to_excel(pwd + '\data3_clean_survey_output.xlsx', index=False)